In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import ta

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LeakyReLU

In [2]:
### DUDAS


# nuestro input es el window size para el LSTM y la cantidad de caracteristicas? diferencia entre las convolusionales 2D?
# si tenemos 2000 datos por feature, cuantas neuronas tenemos de input? 2000? nfeatures segun yo o mas bien, como relaciono el input shape con las neuronas?
# segun yo cada feature representaba una neurona para la capa inicial, o es cada dato, o es independiente?
# lo anterior como sería con convolusionales? los pixeles en convolusionales son features? explicar el por el video que vi
# Para que sirve import tqdm?
# el mismo espacio de tiempo
# mejorar la red neuronal. 
# Usar semillas para el random?


In [3]:
def download_data(ticker: str, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data = data[['Adj Close']]  
    return data


def preprocess_data(data: pd.DataFrame):
    prices = data
    r = (np.log(prices[['Adj Close']]/prices[['Adj Close']].shift(1))).dropna()
    mean = r.mean()
    std = r.std()
    r_norm = (r - mean)/std 
    
    return prices, r, r_norm, mean, std


def generator(data: pd.DataFrame):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(data.shape[0], 1)))
    model.add(LeakyReLU(negative_slope=0.2))
    model.add(LSTM(64))
    model.add(LeakyReLU(negative_slope=0.2) )
    model.add(Dense(252))
    
    return model

def discriminator():
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(252, 1))) ## (252, 1)
    model.add(LeakyReLU(negative_slope=0.2))
    model.add(LSTM(100))
    model.add(LeakyReLU(negative_slope=0.2))
    model.add(Dense(1, activation='sigmoid')) 

    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model


@tf.function
def train_step(data, batch_size = 100):
    noise = tf.random.normal([batch_size, len(data), 1])
    
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_data = gen_model(noise, training = True)
        
        y_real = disc_model(data, training = True)
        y_fake = disc_model(generated_data, training = True)
        
        gen_loss = -tf.math.reduce_mean(y_fake) # o simplemente -tf.math.reduce_mean(y_fake) y sin las funciones de gen_loss y disc_loss
        disc_loss = tf.reduce_mean(y_fake) - tf.reduce_mean(y_real) #o simplemente tf.reduce_mean(y_fake) - tf.reduce_mean(y_real)
        
        
    gradients_gen = gen_tape.gradient(gen_loss, gen_model.trainable_variables)
    gradients_disc = disc_tape.gradient(disc_loss, disc_model.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_gen, gen_model.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_disc, disc_model.trainable_variables))
    
    
    return gen_loss, disc_loss



class Position:
    def __init__(self, ticker: str, price: float, n_shares: int):
        self.ticker = ticker
        self.price = price
        self.n_shares = n_shares
        
        
def backtest(data: pd.DataFrame, sl: float, tp: float,
             n_shares: int, rf = 0):
    
    
    data = data.copy()

    data.columns.values[0] = "Close"
    
    bollinger = ta.volatility.BollingerBands(data.Close, window=20)
    data['BB_Buy'] = bollinger.bollinger_lband_indicator()

    

    capital = 1_000_000
    COM = 0.125 / 100  # Commission percentage
    active_long_positions = []
    portfolio_value = [capital]
     
    
    wins = 0
    losses = 0

    # Iterar sobre los datos del mercado
    for i, row in data.iterrows():
        long_signal = row.BB_Buy  # Señal de compra

        # Entrada de posición larga
        if long_signal == True:
            cost = row.Close * n_shares * (1 + COM)
            if capital > cost and len(active_long_positions) < 100:
                capital -= row.Close * n_shares * (1 + COM)
                active_long_positions.append(
                    Position(ticker="MANU", price=row.Close, n_shares=n_shares))

      

        # Cierre de posiciones largas
        for position in active_long_positions.copy():
            if row.Close > position.price * (1 + tp):
                capital += row.Close * position.n_shares * (1 - COM)
                wins += 1  # Operación ganadora
                active_long_positions.remove(position)
            elif row.Close < position.price * (1 - sl):
                capital += row.Close * position.n_shares * (1 - COM)
                losses += 1  # Operación perdedora
                active_long_positions.remove(position)
        
        value = capital + len(active_long_positions) * n_shares * row.Close       
        portfolio_value.append(value)


    # Convertir portfolio_value a una Serie de pandas
    portfolio_series = pd.Series(portfolio_value)

    # Calcular el rendimiento logarítmico
    portafolio_value_rends = np.log(portfolio_series / portfolio_series.shift(1))
    
        # Calcular el Sharpe Ratio
    mean_portfolio_return = portafolio_value_rends.mean()  # Rendimiento promedio del portafolio
    portfolio_volatility = portafolio_value_rends.std()  # Volatilidad del portafolio
    sharpe_ratio = (mean_portfolio_return - rf) / portfolio_volatility  # Sharpe Ratio

    #print(f"Sharpe Ratio: {sharpe_ratio:.4f}")

    # Calcular el valor máximo acumulado en cada momento
    running_max = portfolio_series.cummax()

    # Calcular el Drawdown
    drawdown = (portfolio_series - running_max) / running_max

    # Max Drawdown
    max_drawdown = drawdown.min()

    #print(f"Max Drawdown: {max_drawdown:.4f}")

    # Calcular el Win-Loss Ratio
    if losses > 0:
        win_loss_ratio = wins / losses
    else:
        win_loss_ratio = np.inf  # Si no hay pérdidas, el Win-Loss ratio es infinito

    passive = list(data.Close)

    #print(f"Win-Loss Ratio: {win_loss_ratio:.2f}")


    calmar_value = calmar_ratio(portafolio_value_rends)
    

    return calmar_value, portfolio_series



def calmar_ratio(returns):
    # Calcula el retorno anualizado
    annual_return = np.mean(returns) * 252  # 252 es el número promedio de días de mercado en un año
    
    # Calcula el drawdown máximo
    cumulative_returns = (1 + returns).cumprod()  # Retorno acumulado
    peak = cumulative_returns.cummax()  # Punto más alto
    drawdown = (cumulative_returns - peak) / peak  # Pérdida desde el pico
    max_drawdown = drawdown.min()  # Drawdown máximo
    
    # Calcula el Calmar Ratio
    calmar_ratio = annual_return / abs(max_drawdown)
    
    return calmar_ratio



generator_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001)
discriminator_optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

In [4]:
ticker = "MANU"  
start_date = '2014-10-29'
end_date = '2024-10-30'
data = download_data(ticker, start_date, end_date)
precios, data, data_norm, mean, std = preprocess_data(data)
gen_model = generator(data_norm)
disc_model = discriminator()

[*********************100%%**********************]  1 of 1 completed
C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
gen_loss_history = []
disc_loss_history = []

num_batches = data_norm.shape[0] // 200
for epoch in range(100):
    for i in tqdm.tqdm(range(num_batches)):
        batch = data_norm[i*200:(1+i)*200]
        gen_loss, disc_loss = train_step(batch)

        gen_loss_history.append(gen_loss.numpy())
        disc_loss_history.append(disc_loss.numpy())

  8%|▊         | 1/12 [00:01<00:18,  1.70s/it]

In [ ]:
plt.plot(gen_loss_history) # se tiene que acercar a cero. por que?

In [ ]:
plt.plot(disc_loss_history) # se tiene que alejar mas. por que?

In [ ]:
plt.plot(gen_loss_history)
plt.plot(disc_loss_history)

In [ ]:
noise = tf.random.normal([100, 2000, 1])  

generated_series = gen_model(noise, training=False)  

plt.figure(figsize=(12, 6))
for j in range(100):  
    plt.plot(generated_series[j, :])

plt.title("Rendientos generadas")
plt.xlabel("Tiempos")
plt.ylabel("Valores de rendimiento")
plt.legend()
plt.show()

In [ ]:
scenarios = generated_series.numpy().tolist()

In [ ]:
S0 = precios['Adj Close'].sample(n=1).iloc[0]

data_n = []

for scenario in scenarios:
    prices = [S0]
    for log_return in scenario:
        next_price = prices[-1] * np.exp(log_return)
        prices.append(next_price)
    data_n.append(prices)

for prices in data_n:
    plt.plot(prices, alpha=0.5, linewidth=0.75)

plt.plot((precios[1760:]).values, label='Real Price', color='black', linewidth=1.5)
plt.title('Simulated Prices vs Real Price')
plt.ylabel('Price')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
scenarios_df = pd.DataFrame()
for i in range(len(data_n)):
    scenarios_df[f'Simulación {i+1}'] = data_n[i]
    
scenarios_df['Close'] = precios['Adj Close'].iloc[:253].values  # agregar precio original
scenarios_df

In [ ]:
# Solo 10 combinaciones de sl y tp
combinations = [
    (0.01, 0.02),
    (0.01, 0.05),
    (0.01, 0.08),
    (0.02, 0.02),
    (0.02, 0.05),
    (0.02, 0.08),
    (0.03, 0.02),
    (0.03, 0.05),
    (0.03, 0.08),
    (0.025, 0.025)  # Agrega cualquier combinación adicional específica
]

results = []

# Loop para cada combinación en la lista predefinida
for sl, tp in combinations:
    calmar_ratios = []

    # Ejecuta 10 simulaciones por combinación
    num_simulations = len(scenarios)
    for i in range(num_simulations):
        # Ejecuta el backtest con la combinación actual
        calmar, _ = backtest(scenarios_df.iloc[:, [i]], sl=sl, tp=tp, n_shares=20)
        calmar_ratios.append(calmar)

    # Calcula la media del Calmar Ratio para esta combinación
    mean_calmar_ratio = np.mean(calmar_ratios)

    # Guarda los resultados
    results.append({
        "sl": sl,
        "tp": tp,
        "mean_calmar_ratio": mean_calmar_ratio
    })

# Imprimir los resultados
for result in results:
    print(f"SL: {result['sl']}, TP: {result['tp']}, Media del Calmar Ratio: {result['mean_calmar_ratio']:.4f}")


# el bollinger se aplica a cada serie o solo 


generated_series[0]

In [ ]:
gen_model.save("./generador.keras")

In [ ]:
gen2 = tf.keras.models.load_model("./generador.keras")